In [2]:
import numpy as np
import meshplot as mp
import open3d as o3d
import torch

%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [14]:
def obb_from_axis(points: np.ndarray, axis_idx: int):
    """get the oriented bounding box from a set of points and a pre-defined axis"""
    # Compute the centroid, points shape: (N, 3)
    centroid = np.mean(points, axis=0)
    # Align points with the fixed axis idx ([1, 0, 0]), so ignore x-coordinates
    if axis_idx == 0:
        points_aligned = points[:, 1:]
        axis_1 = np.array([1, 0, 0])
    elif axis_idx == 1:
        points_aligned = points[:, [0, 2]]
        axis_1 = np.array([0, 1, 0])
    elif axis_idx == 2:
        points_aligned = points[:, :2]
        axis_1 = np.array([0, 0, 1])
    else:  
        raise ValueError(f"axis_idx {axis_idx} not supported!") 

    # Compute PCA on the aligned points
    points_centered = points_aligned - np.mean(points_aligned, axis=0)  
    cov = np.cov(points_centered.T)
    _, vh = np.linalg.eig(cov)
    axis_2, axis_3 = vh[:, 0], vh[:, 1] # 2D!!
    # axis_2, axis_3 = vh[0], vh[1] # 2D!! 
    axis_2, axis_3 = np.round(axis_2, 1), np.round(axis_3, 1)  
    x2, y2 = axis_2
    x3, y3 = axis_3 
    
    if sum(axis_2 < 0) == 2 or (sum(axis_2 < 0) == 1 and sum(axis_2 == 0) == 1):
        axis_2 = -axis_2
    if sum(axis_3 < 0) == 2 or (sum(axis_3 < 0) == 1 and sum(axis_3 == 0) == 1):
        axis_3 = -axis_3

    # remove -0
    axis_2 = np.array([0. if x == -0. else x for x in axis_2])
    axis_3 = np.array([0. if x == -0. else x for x in axis_3]) 
    if axis_idx == 0:
        evec = np.array([
            axis_1,
            [0, axis_2[0], axis_2[1]],
            [0, axis_3[0], axis_3[1]]
            ]).T
    elif axis_idx == 1:
        evec = np.array([
            [axis_2[0], 0, axis_2[1]],
            axis_1,
            [axis_3[0], 0, axis_3[1]]
            ]).T 
    elif axis_idx == 2:
        evec = np.array([
            [axis_2[0], axis_2[1], 0],
            [axis_3[0], axis_3[1], 0],
            axis_1,
            ]).T 
    # Use these axes to find the extents of the OBB
    # # Project points onto these axes 
    all_centered = points - centroid # (N, 3)
    projection = all_centered @ evec # (N, 3) @ (3, 3) -> (N, 3)

    # Find min and max projections to get the extents
    _min = np.min(projection, axis=0)
    _max = np.max(projection, axis=0)
    extent = (_max - _min) # / 2 -> o3d takes full length
    # Construct the OBB using the centroid, axes, and extents 
 
    return dict(center=centroid, R=evec, extent=extent)

def get_handcraft_obb(mesh, z_weight=1.5):
    all_obbs = []
    if isinstance(mesh, np.ndarray):
        vertices = mesh    
    else:
        mesh.remove_unreferenced_vertices()
        mesh.remove_degenerate_faces() 
        vertices = np.array(mesh.vertices) 
    if len(vertices) == 0:
        return dict(center=np.zeros(3), R=np.eye(3), extent=np.ones(3))
    for axis_idx in range(3):
        obb_dict = obb_from_axis(vertices, axis_idx)
        all_obbs.append(obb_dict)

    # select obb with smallest volume, but prioritize axis z 
    bbox_sizes = [np.prod(x['extent']) for x in all_obbs] 
    bbox_sizes[2] /= z_weight # prioritize z axis 
    min_size_idx  = np.argmin(bbox_sizes)
    obb_dict = all_obbs[min_size_idx]
    return obb_dict

def convert_ply_to_voxel_detailed(ply_path):
    
    # Read the point cloud
    pcd = o3d.io.read_point_cloud(ply_path)
    
    # Convert to numpy array
    scaled_points = np.asarray(pcd.points)

    obb_dict = get_handcraft_obb(scaled_points)

    center = torch.tensor(obb_dict['center']).cuda()
    extent = torch.tensor(obb_dict['extent']).cuda()
    R = torch.tensor(obb_dict['R']).cuda()
    scaled_points = (torch.from_numpy(scaled_points).cuda() - center) @ R

    return scaled_points

In [27]:
def standardize_pointcloud(points, output_size=64):

    points = np.asarray(points)
    
    min_coords = points.min(axis=0)
    max_coords = points.max(axis=0)
    
    current_scale = max_coords - min_coords
    
    current_scale = np.where(current_scale == 0, 1e-6, current_scale)
    
    normalized_points = (points - min_coords) / current_scale
    
    scaled_points = normalized_points * (output_size - 1)
    
    voxel_points = np.round(scaled_points).astype(int)
    
    voxel_grid = np.zeros((output_size, output_size, output_size), dtype=bool)
    
    for x, y, z in voxel_points:
        if (0 <= x < output_size and 
            0 <= y < output_size and 
            0 <= z < output_size):
            voxel_grid[x, y, z] = True
    
    return voxel_grid


In [32]:

def save_tensor_to_voxel(voxels):
    grid = voxels

    # Create new voxel grid object and set voxel_size to some value
    # --> otherwise it will default to 0 and the grid will be invisible
    voxel_grid = o3d.geometry.VoxelGrid()
    voxel_grid.voxel_size = 1
    # Iterate over numpy grid
    for z in range(grid.shape[2]):
        for y in range(grid.shape[1]):
            for x in range(grid.shape[0]):
                if grid[x, y, z] == 0:
                    continue
                # Create a voxel object
                voxel = o3d.geometry.Voxel()
                voxel.color = np.array([0.0, 0.0, 0.0]) * (1.0 - grid[x, y, z])
                voxel.grid_index = np.array([x, y, z])
                # Add voxel object to grid
                voxel_grid.add_voxel(voxel)
    o3d.io.write_voxel_grid("voxels1.ply", voxel_grid)

In [35]:
path = "/store/real/ehliang/r2c2r_blender_data_2/r2c2r_data/test/StorageFurniture/44781/loop_0/link_0.ply"
scaled_points = convert_ply_to_voxel_detailed(path)
visualize = scaled_points.detach().cpu().numpy()
mp.plot(visualize, shading={'point_size': 0.03})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0246695…

In [33]:
standardized = standardize_pointcloud(visualize)
save_tensor_to_voxel(standardized)